<a href="https://colab.research.google.com/github/HelenSaraAlex/T-C-Manual-FAQ-Chatbot/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install transformers

from transformers import BertTokenizer, BertForQuestionAnswering

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def answer_question(question, context):
  """
  Answers a question given a context using the pre-trained model.

  Args:
      question (str): The question to be answered.
      context (str): The context in which to find the answer.

  Returns:
      str: The answer to the question, or an empty string if no answer is found.
  """

  inputs = tokenizer(question, context, return_tensors="pt")

  outputs = model(**inputs)
  start_logits = outputs.start_logits.squeeze().tolist()
  end_logits = outputs.end_logits.squeeze().tolist()

  best_start_index = max(range(len(start_logits)), key=start_logits.__getitem__)
  best_end_index = max(range(len(end_logits)), key=end_logits.__getitem__)

  if start_logits[best_start_index] < 0.1 or end_logits[best_end_index] < 0.1:
    return ""

  answer = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][best_start_index:best_end_index+1])

  answer = " ".join(answer)

  return answer

while True:
  context= """
  the saturn is a planet that have rings and jupiter is the largest planet
  """
  question = input("Ask your question (or 'q' to exit): ")

  if question.lower() == 'q':
    print("Exiting question answering session.")
    break

  if not context:
    context = input("Provide context (optional, press Enter if not needed): ")

  answer = answer_question(question, context)
  if answer:
    print("Answer:", answer)
  else:
    print("No answer found in the provided context.")

Ask your question (or 'q' to exit): what is saturn?
Answer: a planet that have rings
Ask your question (or 'q' to exit): jupiter
Answer: saturn
Ask your question (or 'q' to exit): what is jupiter
Answer: jupiter is the largest planet
Ask your question (or 'q' to exit): q
Exiting question answering session.
